## Text to MySQL Queries

In [1]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool


llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-02-12T21:16:17.2401378Z', 'done': True, 'done_reason': 'stop', 'total_duration': 542001400, 'load_duration': 37101100, 'prompt_eval_count': 26, 'prompt_eval_duration': 85000000, 'eval_count': 10, 'eval_duration': 419000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-aad59641-49b4-4578-a9e5-4efdf44d7eda-0', usage_metadata={'input_tokens': 26, 'output_tokens': 10, 'total_tokens': 36})

In [2]:
# !pip install pymysql

from langchain_community.utilities import SQLDatabase
import warnings
warnings.filterwarnings("ignore")

db_path = "db/rma.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")




In [3]:
db.dialect
db.get_usable_table_names()

db.run("SELECT * FROM rma_status LIMIT 5")

"[(1, 'Pending'), (2, 'Approved'), (3, 'Rejected'), (4, 'Processed')]"

### SQL Query Chains

- convert the question into a SQL query;
- execute the query;
- use the result to answer the original question.

In [4]:
from langchain.chains import create_sql_query_chain

sql_chain = create_sql_query_chain(llm, db)

In [5]:
sql_chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [6]:
sql_chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the

In [9]:
question = "Quais são os status dos rmas?"
response = sql_chain.invoke({'question': question})
print(response)

Question: Quais são os status dos rmas?

SQLQuery: 
SELECT DISTINCT status_name FROM rma_status;

Answer: Os três status disponíveis para RMA (Return Merchandise Authorization) são Pending, Approved e Rejected.


### Extract Correct Query from LLM

In [10]:
from scripts.llm import ask_llm
from langchain_core.runnables import chain

In [11]:
@chain
def get_correct_sql_query(input):
    context = input['context']
    question = input['question']

    intruction = """
        Use above context to fetch the correct SQL query for following question
        {}

        Do not enclose query in ```sql and do not write preamble and explanation.
        You MUST return only single SQL query.
    """.format(question)

    response = ask_llm(context=context, question=intruction)

    return response


In [12]:
response = get_correct_sql_query.invoke({'context': response, 'question': question})

In [13]:
db.run(response)

"[('Pending',), ('Approved',), ('Rejected',), ('Processed',)]"

### Final SQL Query Chain 

In [14]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [15]:
execute_query = QuerySQLDataBaseTool(db=db)
sql_query = create_sql_query_chain(llm, db)

final_chain = (
    {'context': sql_query, 'question': RunnablePassthrough()}
    | get_correct_sql_query
    | execute_query
)

In [16]:
question = "Quais são os status dos rmas?"

response = final_chain.invoke({'question': question})
print(response)

[('Approved',), ('Pending',), ('Processed',), ('Rejected',)]
